In [1]:
from platform import python_version

print(python_version())

3.10.2


In [2]:
import pandas as pd

import pubchempy as pcp
import numpy as np
def isNaN(string):
    return string != string
import os
import glob
import re
from pybatchclassyfire import *
import csv 
import time
import json
from pandas import json_normalize
import wget
import string
import urllib.parse
import openpyxl
import statistics
import sys

In [3]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem import PandasTools

INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions


In [8]:
def chemMN(input_dir, resultcsv):
    #read csv
    df = pd.read_csv(resultcsv)
    
    # define empty variable
    dbn= []

    # check the result csv
    for i, row in df.iterrows():
        # to compare each element with each opther element
        for j, row in df.iterrows():

            # if its not same id
            if df['SMILES'][i] != df['SMILES'][j]:

                if not isNaN(df['SMILES'][i]):
                    if not isNaN(df['SMILES'][j]):

                        try:
                            ms = [Chem.MolFromSmiles(df['SMILES'][i]), Chem.MolFromSmiles(df['SMILES'][j])]
                            fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
                            tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
                            dbn.append({
                                'Name_i':df['id_X'][i],
                                'Name_j':df['id_X'][j],
                                'i': df['SMILES'][i],
                                'j': df['SMILES'][j],
                                'Tanimoto': tn
                            })
                        except Exception as e:
                            print(i)
                            print(j)
                            print(e)
    # save chemical similarities                    
    db_edgenode = pd.DataFrame(dbn)

    dfe = []
    heavy_atoms = ['C', 'N', 'P', 'O', 'S']
    for i, row in db_edgenode.iterrows():        
        if 1.0 > db_edgenode['Tanimoto'][i] >= 0.70:
            # list of mol used to calaculate the MCSS
            n = [Chem.MolFromSmiles(db_edgenode['i'][i]),Chem.MolFromSmiles(db_edgenode['j'][i])]
            res = rdFMCS.FindMCS(n)
            sm_res = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))
            # Check if the MCSS has one of the heavy atoms and whether they are
            # more than 3
            elem = [ele for ele in heavy_atoms if(ele in sm_res)]
            if elem and len(sm_res)>=3:
                MCSS_SMILES = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))

            dfe.append({
                'Start':db_edgenode['Name_i'][i],
                'End':db_edgenode['Name_j'][i],
                'Tanimoto':db_edgenode['Tanimoto'][i],
                'Start_SMILES':i,
                'End_SMILES':j,
                'MCSS': MCSS_SMILES
            })

    df_edge = pd.DataFrame(dfe)
    df_edge['Start'] = df_edge['Start'].astype(str)
    df_edge['End'] = df_edge['End'].astype(str)
    df_edge['sorted_row'] = [sorted([a,b]) for a,b in zip(df_edge.Start,df_edge.End)]
    df_edge['sorted_row'] = df_edge['sorted_row'].astype(str)
    df_edge.drop_duplicates(subset=['sorted_row'], inplace=True)

    nodes= []
    for i, row in df.iterrows():
        n = df['id_X'][i]
        nodes.append({
            'nodes':n
        })

    node= pd.DataFrame(nodes)
    
    
    df_edge.to_csv(input_dir + "/MetabolomicsResults/ChemMNedges.tsv", sep='\t')
    node.to_csv(input_dir + "/MetabolomicsResults/ChemMNnodes.csv", index = False)

    newdf = df_edge
    newdf['StartAtt']=np.nan
    newdf['EndAtt']=np.nan
    for i, row in newdf.iterrows():
        for j, row in df.iterrows():
            if newdf['Start'][i]==df['id_X'][j]:
                newdf.loc[i, 'StartAtt'] = df['class'][j]
            if newdf['End'][i]==df['id_X'][j]:
                newdf.loc[i, 'EndAtt'] = df['class'][j]
    newdf.to_csv(input_dir + "/MetabolomicsResults/ChemMNcys.tsv", sep='\t')
    
    return(newdf)

    

In [9]:
newdf = chemMN(input_dir = "/Users/mahnoorzulfiqar/OneDriveUNI/MZML", 
            resultcsv = "/Users/mahnoorzulfiqar/OneDriveUNI/MZML/MetabolomicsResults/Final_Candidate_List.csv")

RDKit ERROR: [20:23:58] Explicit valence for atom # 3 Cl, 3, is greater than permitted
[20:23:58] Explicit valence for atom # 3 Cl, 3, is greater than permitted
RDKit ERROR: [20:23:58] Explicit valence for atom # 1 Cl, 3, is greater than permitted
[20:23:58] Explicit valence for atom # 1 Cl, 3, is greater than permitted


851
1315
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
851
1352
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)


RDKit ERROR: [20:23:58] Explicit valence for atom # 3 Cl, 3, is greater than permitted
[20:23:58] Explicit valence for atom # 3 Cl, 3, is greater than permitted
RDKit ERROR: [20:23:58] Explicit valence for atom # 1 Cl, 3, is greater than permitted
[20:23:58] Explicit valence for atom # 1 Cl, 3, is greater than permitted


852
1315
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
852
1352
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)


RDKit ERROR: [20:23:59] Explicit valence for atom # 3 Cl, 3, is greater than permitted
[20:23:59] Explicit valence for atom # 3 Cl, 3, is greater than permitted
RDKit ERROR: [20:23:59] Explicit valence for atom # 1 Cl, 3, is greater than permitted
[20:23:59] Explicit valence for atom # 1 Cl, 3, is greater than permitted


...

1619
1315
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
1619
1352
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)


RDKit ERROR: [20:28:53] Explicit valence for atom # 3 Cl, 3, is greater than permitted
[20:28:53] Explicit valence for atom # 3 Cl, 3, is greater than permitted
RDKit ERROR: [20:28:53] Explicit valence for atom # 1 Cl, 3, is greater than permitted
[20:28:53] Explicit valence for atom # 1 Cl, 3, is greater than permitted


1620
1315
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
1620
1352
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)


RDKit ERROR: [20:28:54] Explicit valence for atom # 3 Cl, 3, is greater than permitted
[20:28:54] Explicit valence for atom # 3 Cl, 3, is greater than permitted
RDKit ERROR: [20:28:54] Explicit valence for atom # 1 Cl, 3, is greater than permitted
[20:28:54] Explicit valence for atom # 1 Cl, 3, is greater than permitted


1621
1315
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
1621
1352
Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
